# Time optimization in pandas
1. Use preset time format to convert date format
2. Replace for-loop with vectorization


In [1]:
import pandas as pd
import numpy as np

## Datetime

*Usually we run pandas to deal with time, and in most cases many friends are violent problem solving, directly let pandas own conversion and processing. For the usual learning and small test is no problem, but when running some big data is often very slow, and this time performance is in fact completely optimizable.*

In [2]:
%%timeit 
df = pd.read_csv('./dataset/demand_profile.csv')
df['date_time'] = pd.to_datetime(df['date_time'])

915 ms ± 63.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%%timeit 
df = pd.read_csv('./dataset/demand_profile.csv',parse_dates=['date_time'])

877 ms ± 23.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit 
df = pd.read_csv('./dataset/demand_profile.csv')
df['date_time'] = pd.to_datetime(df['date_time'],format='%d/%m/%y %H:%M')

37.3 ms ± 2.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## For-loop
*for-loop is the basic syntax of all programming languages, and beginners are more lazy in order to quickly implement functions. However, it may not be a particularly good choice if we consider the performance of computing time. 
Based on the above data, we now want to add a new feature, but this new feature is generated based on some temporal conditions, varying according to the duration (hours). Below we introduce some methods of implementation and we can see time cost being reduced significantly.*

### Method 1: Crude looping over DataFrame rows using indices

In [5]:
df = pd.read_csv('./dataset/demand_profile.csv')
df['date_time'] = pd.to_datetime(df['date_time'],format='%d/%m/%y %H:%M')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date_time   8760 non-null   datetime64[ns]
 1   energy_kwh  8760 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 137.0 KB


In [7]:
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


In [8]:
# calculate tariff by hour
def apply_tariff(kwh, hour):
    if 0 <= hour < 7:
        rate = 12
    elif 7 <= hour < 17:
        rate = 20
    elif 17 <= hour < 24:
        rate = 28
    else:
        raise ValueError(f'Invalid hour: {hour}')
    return rate * kwh

In [9]:
def apply_tariff_loop(df):
    energy_cost_list = []
    for i in range(len(df)):
        energy_used = df.iloc[i]['energy_kwh']
        hour = df.iloc[i]['date_time'].hour
        energy_cost = apply_tariff(energy_used, hour)
        energy_cost_list.append(energy_cost)
    df['cost_cents'] = energy_cost_list

In [10]:
%timeit apply_tariff_loop(df)

3.37 s ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
df1 = df.copy()
apply_tariff_loop(df1)
df1

,date_time,energy_kwh,cost_cents
0,2013-01-01 00:00:00,0.586,7.032
1,2013-01-01 01:00:00,0.580,6.960
2,2013-01-01 02:00:00,0.572,6.864
3,2013-01-01 03:00:00,0.596,7.152
4,2013-01-01 04:00:00,0.592,7.104
...,...,...,...
8755,2013-12-31 19:00:00,0.450,12.600
8756,2013-12-31 20:00:00,0.543,15.204
8757,2013-12-31 21:00:00,1.135,31.780
8758,2013-12-31 22:00:00,0.729,20.412


### Method 2: Looping with apply()

In [12]:
def apply_tariff_apply(df):
    df['cost_cents'] = df.apply(
        lambda row: apply_tariff(
            kwh=row['energy_kwh'],
            hour=row['date_time'].hour),
        axis=1)

In [13]:
%timeit apply_tariff_apply(df)

164 ms ± 7.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
df2 = df.copy()
apply_tariff_loop(df2)
df2

,date_time,energy_kwh,cost_cents
0,2013-01-01 00:00:00,0.586,7.032
1,2013-01-01 01:00:00,0.580,6.960
2,2013-01-01 02:00:00,0.572,6.864
3,2013-01-01 03:00:00,0.596,7.152
4,2013-01-01 04:00:00,0.592,7.104
...,...,...,...
8755,2013-12-31 19:00:00,0.450,12.600
8756,2013-12-31 20:00:00,0.543,15.204
8757,2013-12-31 21:00:00,1.135,31.780
8758,2013-12-31 22:00:00,0.729,20.412


### Method 3: Vectorization with Pandas series

In [31]:
df.reset_index(inplace=True)

In [32]:
def apply_tariff_isin(df):
    df.set_index('date_time', inplace=True)

    peak_hours = df.index.hour.isin(range(17, 24))
    shoulder_hours = df.index.hour.isin(range(7, 17))
    off_peak_hours = df.index.hour.isin(range(0, 7))

    df.loc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 28
    df.loc[shoulder_hours,'cost_cents'] = df.loc[shoulder_hours, 'energy_kwh'] * 20
    df.loc[off_peak_hours,'cost_cents'] = df.loc[off_peak_hours, 'energy_kwh'] * 12
    
    df.reset_index(inplace=True)

In [33]:
%timeit apply_tariff_isin(df)

5.56 ms ± 141 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


or

In [36]:
def apply_tariff_cut(df):
    df.set_index('date_time', inplace=True)
    cents_per_kwh = pd.cut(x=df.index.hour,
                           bins=[0, 7, 17, 24],
                           include_lowest=True,
                           labels=[12, 20, 28]).astype(int)
    df['cost_cents'] = cents_per_kwh * df['energy_kwh']
    
    df.reset_index(inplace=True)

In [37]:
%timeit apply_tariff_cut(df)

3.31 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
df3 = df.copy()
apply_tariff_loop(df3)
df3

,date_time,energy_kwh,cost_cents
0,2013-01-01 00:00:00,0.586,7.032
1,2013-01-01 01:00:00,0.580,6.960
2,2013-01-01 02:00:00,0.572,6.864
3,2013-01-01 03:00:00,0.596,7.152
4,2013-01-01 04:00:00,0.592,7.104
...,...,...,...
8755,2013-12-31 19:00:00,0.450,12.600
8756,2013-12-31 20:00:00,0.543,15.204
8757,2013-12-31 21:00:00,1.135,31.780
8758,2013-12-31 22:00:00,0.729,20.412


### Method 4: Vectorization with NumPy arrays

In [41]:
def apply_tariff_digitize(df):
    df.set_index('date_time', inplace=True)
    
    prices = np.array([12, 20, 28])
    bins = np.digitize(df.index.hour.values, bins=[7, 17, 24])
    df['cost_cents'] = prices[bins] * df['energy_kwh'].values
    
    df.reset_index(inplace=True)

In [42]:
%timeit apply_tariff_digitize(df)

1.59 ms ± 40.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [43]:
df4 = df.copy()
apply_tariff_loop(df4)
df4

,date_time,energy_kwh,cost_cents
0,2013-01-01 00:00:00,0.586,7.032
1,2013-01-01 01:00:00,0.580,6.960
2,2013-01-01 02:00:00,0.572,6.864
3,2013-01-01 03:00:00,0.596,7.152
4,2013-01-01 04:00:00,0.592,7.104
...,...,...,...
8755,2013-12-31 19:00:00,0.450,12.600
8756,2013-12-31 20:00:00,0.543,15.204
8757,2013-12-31 21:00:00,1.135,31.780
8758,2013-12-31 22:00:00,0.729,20.412


As we can see, the time spent on calculating tariff has been reduced from $3.14s$ to $0.159s$ on average.

For more details, you can visit https://stackoverflow.com/questions/52673285/performance-of-pandas-apply-vs-np-vectorize-to-create-new-column-from-existing-c